# Assignment 1
- **Student name:** Alvaro Alexis Muñoz Reynoso
- **Email:** alvaro.munoz7503@alumnos.udg.mx

## Modules

In [16]:
import numpy as np
import matplotlib.pyplot as plt

import math
from scipy.stats import wrapcauchy
from scipy.stats import cauchy
# from scipy.stats import levy_stable

# from plotly
import plotly.graph_objects as go

import pandas as pd

# Classes

In [9]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)


## Activity 1 Correlated Random Walk 
    - Select rotations from a Cauchy distribution (scipy).
    - Implement usign Vec2d class.
    - Keep trajectory in a numpy array
    - Visualize trajectory using plotly - 2D projection

In [65]:
speed = 5
n_steps = 3000

# 2d 0's numpy matrix generated to save x, y values along the trajectory
trajectory = np.zeros((n_steps, 2))

# Cauchy rvs method generate the rotation angles to be taken on each iteration
cauchy_distribution = cauchy()
rotations = cauchy_distribution.rvs(size=n_steps)

velocity = Vec2d(speed, 0)  # Initial position

for i in range(1, n_steps):
    turn_angle = rotations[i]
    
    velocity = velocity.rotated(turn_angle)
    
    trajectory[i, 0] = trajectory[i-1, 0] + velocity.x
    trajectory[i, 1] = trajectory[i-1, 1] + velocity.y


fig_cauchy = go.Figure()

fig_cauchy.add_trace(go.Scatter(x = trajectory[:, 0],
                                        y = trajectory[:, 1],
                                        marker = dict(size=2),
                                        line = dict(width=2),
                                        mode = 'lines',
                                        name = 'Trajectory',
                                        showlegend = True
                                        
))

fig_cauchy.update_layout(
    title = 'Cauchy trajectory using numpy and Vec2d',
    autosize = False,
    width = 1200,
    height= 600
)
fig_cauchy.show()